In [3]:
import cv2
import numpy as np
from scipy.optimize import minimize

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Unable to read image from {image_path}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use edge detection with adjusted parameters
    edges = cv2.Canny(gray, 30, 100)
    
    # Dilate the edges to close gaps
    kernel = np.ones((5,5), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Display intermediate results
    cv2.imshow("Gray", gray)
    cv2.imshow("Edges", edges)
    cv2.imshow("Dilated", dilated)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return dilated, image

def extract_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def extract_points_from_contours(contours):
    points = []
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        points.extend(approx.reshape(-1, 2))
    return np.array(points)

def calculate_centroid(points):
    return np.mean(points, axis=0)

def detect_symmetry(points):
    if len(points) == 0:
        return []
    
    symmetries = []
    centroid = calculate_centroid(points)
    
    # Check vertical symmetry
    reflected_points = reflect_points(points, [centroid[0], 0], np.inf)
    if np.allclose(reflected_points, points, atol=2):
        symmetries.append("Vertical")
    
    # Check horizontal symmetry
    reflected_points = reflect_points(points, [0, centroid[1]], 0)
    if np.allclose(reflected_points, points, atol=2):
        symmetries.append("Horizontal")
    
    # Check diagonal symmetries (45 and 135 degrees)
    for angle in [np.pi/4, 3*np.pi/4]:
        axis_slope = np.tan(angle)
        reflected_points = reflect_points(points, centroid, axis_slope)
        if np.allclose(reflected_points, points, atol=2):
            symmetries.append(f"Diagonal ({angle*180/np.pi:.0f} degrees)")
    
    return symmetries

def reflect_points(points, axis_point, axis_slope):
    reflected_points = []
    for point in points:
        x, y = point
        a, b = axis_point
        
        if axis_slope == np.inf: 
            x_ref = 2 * a - x
            y_ref = y
        else:
            m = axis_slope
            c = b - m * a

            x_ref = ((1 - m*2) * x + 2 * m * (y - c)) / (1 + m*2) + 2 * a
            y_ref = (2 * m * (x - a) - (1 - m*2) * (y - c)) / (1 + m*2) + c
        
        reflected_points.append([x_ref, y_ref])
    return np.array(reflected_points)

def visualize_contours(image, contours):
    contour_image = image.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
    cv2.imshow("Contours", contour_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def visualize_results(image, points, symmetries, bezier_params):
    for symmetry in symmetries:
        if "Vertical" in symmetry:
            cv2.line(image, (int(image.shape[1]/2), 0), (int(image.shape[1]/2), image.shape[0]), (0, 255, 0), 2)
        elif "Horizontal" in symmetry:
            cv2.line(image, (0, int(image.shape[0]/2)), (image.shape[1], int(image.shape[0]/2)), (0, 255, 0), 2)
        elif "Diagonal" in symmetry:
            angle = float(symmetry.split("(")[1].split()[0])
            if angle == 45:
                cv2.line(image, (0, 0), (image.shape[1], image.shape[0]), (0, 255, 0), 2)
            else:
                cv2.line(image, (image.shape[1], 0), (0, image.shape[0]), (0, 255, 0), 2)

    cv2.imshow("Results", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

try:
    image_path = r'D:\Adobe-gen\problems\problems\occlusion1_rec.png'
    binary_image, original_image = preprocess_image(image_path)
    contours = extract_contours(binary_image)
    visualize_contours(original_image, contours)
    points = extract_points_from_contours(contours)

    if len(points) == 0:
        print("No points detected in the image. Please check the image and preprocessing steps.")
    else:
        symmetries = detect_symmetry(points)

        if symmetries:
            print("Symmetries detected:", symmetries)
            visualize_results(original_image, points, symmetries, None)
        else:
            print("No symmetry detected.")

except Exception as e:
    print(f"An error occurred: {str(e)}")

No symmetry detected.
